In [1]:
from statsbombpy import sb

In [155]:
import pandas as pd

## Data Preparation
We want to build a dashboard that shows data on Lionel Messi's time in Barcelona. The data we are using is from [statsbomb](https://statsbomb.com/news/statsbomb-release-free-messi-data-all-seasons-from-2004-05-2020-21-now-available/). However, the data is very detailed and contains every pass, shot, dribble and more statistics of all Barcelona games from 2004/05 to 2020/21. Therefore, we will process this data to get stats about Messi's goals, assists, minutes played etc. Our processed dataset will be used to create a [Tableau dashboard](https://public.tableau.com/views/MessiinLaLiga/Dashboard1?:language=en-EN&publish=yes&:display_count=n&:origin=viz_share_link).

In [163]:
def detect_goals(df, player):
    '''
    input: df -> full game df, player (str)
    output: number of goals (int), assists_by (dict: {player: assists})
    '''

    goals = df[(df["shot_outcome"]=="Goal") & (df["player"]==player)]
    num_goals = goals.shape[0]
    
    assist_ids = goals["shot_key_pass_id"]
    assists_by = dict(df[df["id"].isin(assist_ids)]["player"].value_counts())
    
    return num_goals, assists_by

In [160]:
def detect_assists(df, player):
    '''
    input: df -> full game df, player (str)
    output: number of assists (int), assists_for (dict: {player: assists})
    '''  
    if not "pass_goal_assist" in df.columns:
        return 0, {}
    
    assists = df[(df["pass_goal_assist"]==True) & (df["player"]==player)]
    num_assists = assists.shape[0]

    goal_ids = assists["pass_assisted_shot_id"]
    assists_for = dict(df[df["id"].isin(goal_ids)]["player"].value_counts())
    
    
    return num_assists, assists_for

In [162]:
def detect_nutmegs(df, player):
    '''
    input: df -> full game df, player (str)
    output: number of nutmegs (int)
    '''
    if 'dribble_nutmeg' in df.columns:
        return df[(df["dribble_nutmeg"]==True) & (df["player"]==player)].shape[0]
    else:
        return 0

In [161]:
def detect_minutes_played(df, player, team):
    '''
    input: df -> full game df, player (str), team (str)
    output: minutes played (int)
    '''
    # check if player in starting XI
    player_started = False
    starting = df[(df["type"]=="Starting XI") & (df["team"]==team)]["tactics"].item()["lineup"]
    for starter in starting:
        if starter["player"]["name"] == player:
            player_started = True
            start_minute = 0
            break
            
    # check if player came from the bench
    if player_started == False:
        sub = df[df["substitution_replacement"]==player]
        if sub.shape[0] > 0:
            start_minute = df[df["substitution_replacement"]==player]["minute"].item()
        else:
            player_played = False
            return 0
    
    # check if player was replaced
    subbed_out = df[(df["substitution_replacement"].notnull()) & df["player"]==player]
    if subbed_out.shape[0] > 0:
        end_minute = subbed_out["minute"].item()
    else:
        end_minute = 90
        
    # calculate minutes played
    minutes_played = end_minute - start_minute
    return minutes_played

In [165]:
def collect_data(player, competition, team):
    '''
    input: player (str), competition (str), team (str)
    output: dataframe with rows for every game the team played in the competition (that is available in the database)
    '''
    df = pd.DataFrame()
    # df.columns = ["season_id", "match_id", "opponent", "num_goals", "assists_by", "num_assists", "assists_for", "num_nutmegs", "minutes_played"]
    
    # get list of available season IDs of the competition
    competitions = sb.competitions()
    seasons = competitions[competitions["competition_name"]==competition]
    competition_id = seasons["competition_id"].unique().item()
    season_ids = list(seasons["season_id"])
    
    for season_id in season_ids:
        print("Starting with season: ", season_id)
        match_ids = list(sb.matches(competition_id=competition_id, season_id=season_id)["match_id"])
        for match_id in match_ids:
            print("Starting with match: ", match_id)
            match_dict = {}
            match_df = sb.events(match_id=match_id)
            
            match_dict["season_id"] = season_id
            match_dict["match_id"] = match_id
            teams = list(match_df[:2]["team"])
            teams.remove(team)
            match_dict["opponent"] = teams[0]
            
            match_dict["num_goals"], match_dict["assists_by"] = detect_goals(match_df, player)
            match_dict["num_assists"], match_dict["assists_for"] = detect_assists(match_df, player)
            match_dict["num_nutmegs"] = detect_nutmegs(match_df, player)
            match_dict["minutes_played"] = detect_minutes_played(match_df, player, team)
            print(match_dict)
            df = df.append(match_dict, ignore_index=True)
            
    return df
    

In [167]:
df = collect_data("Lionel Andrés Messi Cuccittini", "La Liga", "Barcelona")

credentials were not supplied. open data access only
Starting with season:  90
credentials were not supplied. open data access only
Starting with match:  3773631
credentials were not supplied. open data access only
{'season_id': 90, 'match_id': 3773631, 'opponent': 'Real Betis', 'num_goals': 1, 'assists_by': {'Ousmane Dembélé': 1}, 'num_assists': 0, 'assists_for': {}, 'num_nutmegs': 0, 'minutes_played': 34}
Starting with match:  3773665
credentials were not supplied. open data access only
{'season_id': 90, 'match_id': 3773665, 'opponent': 'Osasuna', 'num_goals': 0, 'assists_by': {}, 'num_assists': 2, 'assists_for': {'Moriba Kourouma Kourouma': 1, 'Jordi Alba Ramos': 1}, 'num_nutmegs': 0, 'minutes_played': 90}
Starting with match:  3773497
credentials were not supplied. open data access only
{'season_id': 90, 'match_id': 3773497, 'opponent': 'Real Madrid', 'num_goals': 0, 'assists_by': {}, 'num_assists': 0, 'assists_for': {}, 'num_nutmegs': 0, 'minutes_played': 90}
Starting with match: 

In [168]:
df.head()

,assists_by,assists_for,match_id,minutes_played,num_assists,num_goals,num_nutmegs,opponent,season_id
0,{'Ousmane Dembélé': 1},{},3773631.0,34.0,0.0,1.0,0.0,Real Betis,90.0
1,{},"{'Moriba Kourouma Kourouma': 1, 'Jordi Alba Ra...",3773665.0,90.0,2.0,0.0,0.0,Osasuna,90.0
2,{},{},3773497.0,90.0,0.0,0.0,0.0,Real Madrid,90.0
3,{'Frenkie de Jong': 1},{},3773660.0,90.0,0.0,1.0,0.0,Levante,90.0
4,{},{},3773593.0,90.0,0.0,1.0,0.0,Villarreal,90.0


In [ ]:
assists_by_df = pd.DataFrame()
assists_for_df = pd.DataFrame()
for i, row in df.iterrows():
    d = {}
    d["season_id"] = row["season_id"]
    d["match_id"] = row["match_id"]
    d["opponent"] = row["opponent"]
    
    # get assists by
    for player, num_assists in row["assists_by"].items():
        assists_by_dict = d.copy()
        assists_by_dict["player"] = player
        assists_by_dict["num_assists"] = num_assists
        assists_by_df = assists_by_df.append(assists_by_dict, ignore_index=True)
        
    
    # get assists for
    for player, num_assists in row["assists_for"].items():
        assists_for_dict = d.copy()
        assists_for_dict["player"] = player
        assists_for_dict["num_assists"] = num_assists
        assists_for_df = assists_for_df.append(assists_for_dict, ignore_index=True)

In [189]:
df.to_csv("messi_barcelona_games.csv", index=False)
assists_for_df.to_csv("messi_assists.csv", index=False)
assists_by_df.to_csv("messi_assisted.csv", index=False)

[Tableau dashboard](https://public.tableau.com/views/MessiinLaLiga/Dashboard1?:language=en-EN&publish=yes&:display_count=n&:origin=viz_share_link)